# Setup

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
original_image = cv2.imread('test.jpg')

# Converts the image to RGB instead OpenCV's default BGR format for matplotlib
image_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

plt.imshow(image_rgb)
plt.axis('off')
plt.show()

# Edge Detection

In [ ]:
# Converts image to grayscale for easier edge detection
image_gray = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

plt.imshow(image_gray, cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
# Blurs the grayscale image to reduce noise and improve edge detection
image_blurred = cv2.GaussianBlur(image_gray, (5, 5), 0)

plt.imshow(image_blurred, cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
# Uses Canny edge detection algorithm on blurred grayscale image
image_edges = cv2.Canny(image_blurred, 50, 150)

plt.imshow(image_edges, cmap='gray')
plt.axis('off')
plt.show()

# Contour Detection

In [ ]:
# Finds contours from the edges detected
contour, hierarchy = cv2.findContours(image_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Makes a copy of the original image to draw contours on
image_contours = original_image.copy()
cv2.drawContours(image_contours, contour, -1, (255, 0, 255), 2)

plt.imshow(cv2.cvtColor(image_contours, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()